Consolidated data pipeline program. 2 parts (as of Jan. 6, 2019): flat prep, and science image reduction. In previous versions, those two steps are combined, but here (since Gaussian filter needs to be applied to flats after rf-off sub) seperated for flexibility's sake.

Completes the following steps (some are optional):
- Wavelength cal (if data is from before Jan. 2018)
- Doppler shift correction for motion of planet (but not rotation)
- Attaches ephemerides and telescope pointing info
- Can create a pixel mask if you want
- Corrects for hot pixels/cosmic rays
- Divides by exposure time
- Can correct for fringing
- Applies Gaussian filter to noisy flats
- Divides by flats
- Subtracts RF-off images

Before running:
- Have completed science image list, for target and flats. This means, check for saturation, unbinned images, repeats beforehand. No wavelength repeats!
- Have lists of ephemerides downloaded from JPL Horizons. 
- Have APO pointing logfile
- Know if you need a pixel mask made or not. 
- Generally a good idea to saw a directory of the raw data in case of any mistakes

How to run:
1. Compile each subroutine
2. Run flat_prep()
3. Once flats have been reduced, run reduce_datacube()
4. Check data manually to make sure everything worked correctly

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import scipy.ndimage as ndimage
from astropy.io import fits
from scipy.ndimage import filters
from astropy import units as u
from astropy.nddata import CCDData
import ccdproc
import datetime
import math

In [ ]:
# Functions for correcting wavelengths of images taken before January 2018

# Further work in 2019 showed that this correction isn't good and shouldn't be applied, regardless of when images were taken; but, if fringing frames (as developed by Erandi's program) were made using these 'corrected' wavelengths, you'll need to correct the wavelengths associated with the Jupiter and flat frames, since the fringing correction program checks wavelengths.

def RF_function(wl):
    '''Takes a wavelength value of an image, returns the RF value that should be used to generate that wavelength. '''
    A1 = 0.152464324712792 
    A2 = -0.249953566211374 
    A3 = -0.168609021526478 
    A4 = -1.26484658735496 
    A5 = 6.59454398355950 
    A6 = -23.6712903221411 
    A7 = 101.932242630767
    B1 = 724.867720000000
    B2 = 144.671593361344
    x = (wl-B1)/B2 # centering and scaling transformation
    return A1*x**6 + A2*x**5 + A3*x**4 + A4*x**3 + A5*x**2 + A6*x + A7
    
def wl_finder(RF,read_wl):
    '''Use wl and RF frequency array from RF() to interpolate between points and find wl from RF frequency and wl read from image header'''
    #define wavelength space using read wavelength of image (10 nm on either side)
    wl_space = np.linspace(read_wl-40,read_wl+40,10000)
    #define function over that wavelength space
    wl_func = interp1d(RF_function(wl_space),wl_space)
    #return the wavelength at the input RF using interpolated function
    return wl_func(RF)

def wavelength_corrector(imagelist,directory):
    '''Uses wl_finder() and RF_finction() to save actual wavelength to images'''
    images = np.loadtxt(imagelist,dtype=str)
    for i in range(0,len(images)):
        #correcting both rf-off and rf-on images
        im = fits.open(directory+images[i])
        image_wavelength = im[0].header['lambda']
        image_rf = im[0].header['rffreq']
        actual_wavelength = round(float(wl_finder(image_rf,image_wavelength)),3)
    
        #save old and new wavelengths to header
        im[0].header['ORIG_WL'] = (image_wavelength,'Original recorded wl')
        im[0].header['LAMBDA'] = (actual_wavelength,'Corrected wl')
        im.writeto(directory+images[i],overwrite=True)
        im.close()

In [ ]:
def doppler_shift_corr(imagelist,directory):
    '''
    Corrects wavelength for Jupiter's motion (but not its rotation)
    
    Directory = path of location of images
    imlist = path+name of image list
    '''
    c = 3e5 # km/s
    imlist = np.loadtxt(imagelist,dtype=str)
    
    for i in range(0,len(imlist)):
        im = fits.open(directory+imlist[i])
        if im[0].header['rfon'] == 1:
            wl_naught = im[0].header['lambda'] #nm
            v_r = im[0].header['pe_rate'] # km/s
            
            im[0].header['WLPREDOP'] = (wl_naught,'Wavelength before Doppler correction')
            im[0].header['LAMBDA'] = (wl_naught*((v_r/c)+1),'Wavelength (nm)')
            im[0].header['HISTORY'] = ('Wavelength corrected for Doppler shift')
            im.writeto(directory+imlist[i],overwrite=True)
        im.close()

In [ ]:
#Depending on input used to make ephimerides file, can include RA and dec columns, which will affect column indeces. There are two options in the following program that account for these indeces changes. Be sure to check the columns being looked at match what this program is pulling out.

def attach_eph(directory,image_file,eph_file):
    '''
    Attaches Jupiter's ephimerides information to the fits header.
    
    directory: where images to attach eph are located, where images will be saved to 
    image_file: location+name of list of images
    eph_file: location+name of ephemerides file from horizons
    
    Make ephimerides file using https://ssd.jpl.nasa.gov/horizons.cgi
    Settings for Horizons: (From Paul's program 'get_ephemeris.pro')
    ; QUANTITIES= '4,8,11,13,14,15,17,19,20,21,24'
    ;
    ;  4. Apparent Azimuth & Elevation
    ;  8. Airmass
    ; 11. Defect of illumination
    ; 13. Target angular diameter
    ; 14. Obs sub-long & sub-lat
    ; 15. Solar sub-long & sub-lat
    ; 17. N. Pole Pos. Ang & Dis
    ; 19. Helio range & range rate
    ; 20. Observer range & range rate
    ; 21. One-way light travel time
    ; 24. Sun-Target-Observer angle

    ; NOTE: Be sure to have the following table settings:
    ;					Date/Time: Julian Day
    ;					Time Digits: Fractional Seconds
    ;					Output Units: km & km/s
    ;					Range Units: Kilometers
    ;					Refraction Model: AIRLESS
    ;					Extra Precision: YES
    ;					CVS Format: Yes
    '''
    
    eph = np.loadtxt(eph_file,dtype=str,delimiter=',')
    image_list = np.loadtxt(image_file,dtype=str)
    
    #Define empty lists of ephemerides
    jd = []
    RA = [] #hr min s
    dec = [] #deg min s
    az = [] #azimuth, degrees
    alt = [] #altitude, degrees
    airmass = [] #airmass
    ext = [] #mag of extinction
    def_ill = [] #defect of illumination (arcseconds)
    angd = [] #angular diameter (arcseconds)
    obs_long = [] #degrees
    obs_lat = [] #degrees
    sol_long = [] #degrees
    sol_lat = [] #degrees
    np_ang = [] #degrees and arcseconds
    np_dist = [] #degrees and arcseconds
    r = [] #km
    rdot = [] #km/s
    delta_r = [] #km
    delta_rdot = [] #km/s
    lt = [] #light travel time (minutes)
    phase_angle = [] #(degrees)
    
    # make lists of each column: ********Convert data types??
    for i in range(0,len(eph)):
        
        if len(eph[0]) == 21: # if RA and dec weren't included
            ra_dec_index = 0
        elif len(eph[0]) == 23: # if RA and dec are included
            ra_dec_index = 2
            RA.append(eph[i][3])
            dec.append(eph[i][4])
        
        if eph[i][5+ra_dec_index] == '   n.a.': #ignore airmass and the rest of the values if no airmass (Jupiter is below horizon)
            continue
        if eph[i][3+ra_dec_index] == '   n.a.':
            continue
        jd.append(float(eph[i][0]))
        az.append(float(eph[i][3+ra_dec_index]))
        alt.append(float(eph[i][4+ra_dec_index]))
        airmass.append(float(eph[i][5+ra_dec_index]))
        ext.append(float(eph[i][6+ra_dec_index]))
        def_ill.append(float(eph[i][7+ra_dec_index]))
        angd.append(float(eph[i][8+ra_dec_index]))
        obs_long.append(float(eph[i][9+ra_dec_index]))
        obs_lat.append(float(eph[i][10+ra_dec_index]))
        sol_long.append(float(eph[i][11+ra_dec_index]))
        sol_lat.append(float(eph[i][12+ra_dec_index]))
        np_ang.append(float(eph[i][13+ra_dec_index]))
        np_dist.append(float(eph[i][14+ra_dec_index]))
        r.append(float(eph[i][15+ra_dec_index]))
        rdot.append(float(eph[i][16+ra_dec_index]))
        delta_r.append(float(eph[i][17+ra_dec_index]))
        delta_rdot.append(float(eph[i][18+ra_dec_index]))
        lt.append(float(eph[i][19+ra_dec_index]))
        phase_angle.append(float(eph[i][20+ra_dec_index]))
    
    #Interpolate between points, make a function of time for each quantity of interest:
    if ra_dec_index == 2:
        RA_func = interp1d(jd,RA) 
        dec_func = interp1d(jd,dec) 
    az_func = interp1d(jd,az)
    alt_func = interp1d(jd,alt)
    airmass_func = interp1d(jd,airmass)
    ext_func = interp1d(jd,ext)
    def_ill_func = interp1d(jd,def_ill)
    angd_func = interp1d(jd,angd)
    obs_long_func = interp1d(jd,obs_long)
    obs_lat_func = interp1d(jd,obs_lat)
    sol_long_func = interp1d(jd,sol_long)
    sol_lat_func = interp1d(jd,sol_lat)
    np_ang_func = interp1d(jd,np_ang)
    np_dist_func = interp1d(jd,np_dist)
    r_func = interp1d(jd,r)
    rdot_func = interp1d(jd,rdot)
    delta_r_func = interp1d(jd,delta_r)
    delta_rdot_func = interp1d(jd,delta_rdot)
    lt_func = interp1d(jd,lt)
    phase_angle_func = interp1d(jd,phase_angle)
    
    #Look through a list of images
    for i in range(0,len(image_list)):
        im = fits.open(directory+image_list[i])
        header = im[0].header
        
        if header['object'] == '            Jupiter': #!!!Maybe get rid of this? Want to append it no matter what?
            im_jd = header['julian']
            #print 'Appending quantities for image '+image_list[i]+' taken at '+str(im_jd)+'...'
            
            #Interpret each function at im_jd, the time when the Jupiter image in question was taken
            im_az = round(az_func(im_jd),7) #Round the values, since FITS headers can't handle the big floats
            im_alt = round(alt_func(im_jd),7)
            im_airmass = round(airmass_func(im_jd),5)
            im_ext = float(ext_func(im_jd))
            im_def_ill = round(def_ill_func(im_jd),5)
            im_angd = round(angd_func(im_jd),5)
            im_obs_long = round(obs_long_func(im_jd),5)
            im_obs_lat = round(obs_lat_func(im_jd),5)
            im_sol_long = round(sol_long_func(im_jd),5)
            im_sol_lat = round(sol_lat_func(im_jd),5)
            im_np_ang = float(np_ang_func(im_jd))
            im_np_dist = float(np_dist_func(im_jd))
            im_r = round(r_func(im_jd),5)
            im_rdot = round(rdot_func(im_jd),5)
            im_delta_r = round(delta_r_func(im_jd),5)
            im_delta_rdot = round(delta_rdot_func(im_jd),5)
            im_lt = round(lt_func(im_jd),5)
            im_phase_angle = round(phase_angle_func(im_jd),5)
            
            
            #Attach them to the image header
            header['AZI_APP'] = (im_az,'Apparent azimuth (AIRLESS) (degrees)') 
            header['ALT_APP'] = (im_alt, 'Apprent altitude (AIRLESS) (degrees)')
            header['AIRMASS'] = im_airmass
            header['MAG_EXT'] = (im_ext,'magnitudes') #*Paul didn't seem to use this?
            header['ANG_DFCT'] = (im_def_ill,'Defect of illumination (arcseconds)')
            header['ANG_DIAM'] = (im_angd,'Full planet angular diameter (arcseconds)')
            header['SE_LON'] = (im_obs_long,'Long of targ. as seen by obs. System III (deg)')
            header['SE_LAT'] = (im_obs_lat,'Lat of target as seen by obs. Planetodetic (deg)')
            header['SS_LON'] = (im_sol_long,'Long of target as seen by Sun. System III (deg)')
            header['SS_LAT'] = (im_sol_lat,'Lat of targ. as seen by Sun. Planetodetic (deg)')
            header['NPPANGLE'] = (im_np_ang,'NP pos. ang (CCW wrt dir. of CNP) (deg)')
            header['NPANGDIS'] = (im_np_dist,'NP ang dist from center of disk (arcsec)')
            header['PS_NGR'] = (im_r,'Planet-Sun range (km)')
            header['PS_RATE'] = (im_rdot,'Planet-Sun range rate (km/s)')
            header['PE_RNG'] = (im_delta_r,'Planet-Earth range (km)')
            header['PE_RATE'] = (im_delta_rdot,'Planet-Earth range rate (km/s)')
            header['ONEWAYLT'] = (im_lt,'1-way light-time from Jupiter to APO (minutes)')
            header['PHASEANG'] = (im_phase_angle,'Phase angle (degrees)')
            
            #Save image
            im.writeto(directory+image_list[i],overwrite=True)
            im.close()
            
#Looks through a tcc text file (From Russet) and attaches the alt and az of the instrument wrt the sky. 

def attach_apo(directory,images,apolog):
    '''
    Attaches telescope pointing data from log file from APO. Pulls the JD of the image from image_list in directory, interpolates the alt and az of the instrument at that time.
    Email APO 3.5-m Lead Observing Specialist (Russet McMillan, mcmillan@apo.nmsu.edu, as of 2019)
    
    directory: directory where data is
    images: location+name of image list
    apolog: location and name of tel.txt files
    '''
    
    image_list = np.loadtxt(images,dtype=str)
    apo_log = np.loadtxt(apolog,dtype=str)
    
    #Make arrays of the log's alt, az, and JD:
    alt = []
    az = []
    tcc_rot_ang = []
    jd = []
    
    JD_per_s = 1./86400
    JD_per_min = 1./1440
    JD_per_hr = 1./24
    
    #Find the jd of the night; pull it from an image in the list
    juliandate = float(int(fits.open(directory+image_list[1])[0].header['julian']))
    
    #make lists of alt and az from apo log to make into functions:
    for i in range(0,len(apo_log)):
        alt.append(float((apo_log[i][10][:-1])))
        #Since az data is weird:
        if 0 < float((apo_log[i][9][7:-1])) <= 180.:
            az.append(180-float((apo_log[i][9][7:-1]))) 
        elif 180 < float((apo_log[i][9][7:-1])) <= 360.:
            az.append(540-float((apo_log[i][9][7:-1])))
        elif float((apo_log[i][9][7:-1])) < 0:
            az.append(180+abs(float((apo_log[i][9][7:-1]))))
        #from prepare_for_geocal; laking a list of tcc rotation angle.
        #THIS ASSUMES THAT THE TELESCOPE IS RECORDING +CW until 180, then negative from ccw (Seems to be right?)
        #append the value of ObjInsAng if it's positive
        if float(apo_log[i][12][11:-1]) >= 0:
            tcc_rot_ang.append(float(apo_log[i][12][11:-1]))
        #if it's negative:
        if float(apo_log[i][12][11:-1]) < 0:
            tcc_rot_ang.append(360.0+float(apo_log[i][12][11:-1]))
    
        #Convert hours:minutes:seconds into JD, append it
        jd.append(juliandate+0.5+(float(apo_log[i][1][0:2])*JD_per_hr)+(float(apo_log[i][1][3:5])*JD_per_min)+(float(apo_log[i][1][6:-1])*JD_per_s))
    
    #Make functions of alt and az as a function of JD via interp1d
    alt_func = interp1d(jd,alt)
    az_func = interp1d(jd,az)
    rotang_func = interp1d(jd,tcc_rot_ang)
    
    #Open the image and find the JD
    for i in range(0,len(image_list)):
        
        im = fits.open(directory+image_list[i])
        header = im[0].header
        im_jd = header['julian']
        #print 'Appending quantities for image '+image_list[i]+' taken at '+str(im_jd)+'...'
        
        im_alt = round(alt_func(im_jd),6)
        im_az = round(az_func(im_jd),6)
        
        #print im_jd,im_alt,im_az
        
        #Append the alt and az of the instrument at the image's JD
        header['TCC_ALT'] = (im_alt,'Encoder-reported altitude from APO')
        header['TCC_AZ'] = (im_az,'Encoder-reported azimuth from APO')
        
        #appending rot_ang and pnorth, stuff needed for geometric cal
        if header['rfon'] == 1:
            im_jd = header['julian']
            #print 'Appending telescope quantities for image '+image_list[i]+' taken at '+str(im_jd)+'...'

            im_rotang = round(rotang_func(im_jd),5)

            #Append the alt and az of the instrument at the image's JD
            header['ROT_ANG'] = (im_rotang,'(Degrees) angle cw from pos vertical to NCP')

            #Find np_ang
            npa = round(im_rotang + header['NPPANGLE'],5)
            header['PNORTH'] = (npa,'Angle of the planets NP cw from CN (Degrees)')
        
        #Save image:
        im.writeto(directory+image_list[i],overwrite=True)
        im.close()

In [ ]:
# Program to make a hot pixel mask if one hasn't been made yet
def pix_mask_maker(flatlist, directory,pixmask_path_and_name):
    
    '''
    Makes a hot pixel mask from list of flats. If the same pixel is 5 sigma above the average of its neighbors in over half of the flats, flagged as a hot pixel and saved to the pixel mask.
    
    flatlist: path+name of list of flats
    directory: path of location of flats
    pixmask_path_and_name: path+name of where the pixel mask wil be saved.
    Completed pixel masks, on desktop, are currently in ~/NAIC/pixel_masks/
    '''
    
    image_list = np.loadtxt(flatlist,dtype=str)
    hot_pix_list = [] # Big list of all potential 'hot pixels' detected in each image

    # Define the empty pixel mask:
    im = fits.open(directory+image_list[1])
    dimension1 = im[0].header['NAXIS1']
    dimension2 = im[0].header['NAXIS2']
    pix_mask = np.zeros((dimension1,dimension2)) #Empty pixel mask
    im.close()
    
    # Function to apply to neighbors:
    def neighbors(values):
        # Returns mean of neighbors plus 5 std of neighbors for given pixel value
        return np.mean(values)+5*np.std(values)

    # Loop through flats, finding potential hot pixels:
    for i in range(0,len(image_list)):
        im = fits.open(directory+image_list[i])

        # neighbor footprint
        footprint = np.array([[1,1,1],
                              [1,0,1],
                              [1,1,1]])

        # make array of median + 5 std devs of neighbors:
        neighbor_results = ndimage.generic_filter(im[0].data, neighbors, footprint=footprint)
        g,h = np.where(im[0].data > neighbor_results) # positions of hot pixels
        
        # add that image's hot pixels to master list
        for l in range(0,len(g)):
            hot_pix_list.append((g[l],h[l]))
            
        # print out percentage complete
        if i%3 == 0:
            print round(float(i)/len(image_list),4)*100,'% done;',i,'images analyzed'
            
    # Make the pixel mask:
    d = {x:hot_pix_list.count(x) for x in hot_pix_list} # in hot_pix_list, count the times each pair shows up
    pairs = d.keys() #unique x and y coordinates of each hot pixel
    number_of_pairs = d.values() #number of times each x and y coordinate is counted as a hot pixel
    number_of_flats = len(image_list)

    #If the hot pixel shows up in more than half the flats, save position as official hot pixel in the pixel mask
    for i in range(0,len(number_of_pairs)):
        if number_of_pairs[i] >= number_of_flats/2.: 
            pix_mask[pairs[i][0]][pairs[i][1]] = 1

    # Save the pixel mask file
    np.savetxt(pixmask_path_and_name,pix_mask,fmt='%i')

In [ ]:
# Function to correct hot pixels using pixel mask and CR correct 

def pixel_corrector(directory,imagelist,pixelmask,rfon_or_off):
    '''
    Program to correct for cosmic rays and hot pixels. 
    Uses http://ccdproc.readthedocs.io/en/latest/api/ccdproc.cosmicray_lacosmic.html#ccdproc.cosmicray_lacosmic to correct for cosmic rays
    '''
    
    images = np.loadtxt(imagelist,dtype=str)
    pixel_mask = np.loadtxt(pixelmask,dtype=int)
    
    # Function to apply to neighbors:
    def neighbors(values):
        # Returns median of neighbors
        return np.median(values)

    # neighbor footprint
    footprint = np.array([[1,1,1],
                          [1,0,1],
                          [1,1,1]])
    
    for i in range(0,len(images)):
        im = fits.open(directory+images[i])
        if im[0].header['rfon'] == rfon_or_off:
            
            # Replace pixels where pix_mask is non-zero with median of neighbors:
            # array of each pixels' median of its neighbors, same size as image:
            median_array = ndimage.generic_filter(im[0].data, neighbors, footprint=footprint)
            # Set hot pixel locations in image array to their neighbors' median value:
            im[0].data[np.where(pixel_mask > 0)] = median_array[np.where(pixel_mask > 0)]
            
            # Add the locations of cosmic rays as a 2nd HDU before data array gets changed (acess via im[1].data, will be integers)
            im.append(fits.ImageHDU(data = ccdproc.cosmicray_lacosmic(im[0].data,sigclip=7)[1].astype(int),name='CR_MASK'))
            
            #Fix the image, save it to the data array
            im[0].data = ccdproc.cosmicray_lacosmic(im[0].data,sigclip=7)[0]

            #Write history statement, save and close FITS file
            im[0].header['history'] = 'Hot pixels and cosmic rays corrected at '+str(datetime.datetime.now())
            im.writeto(directory+images[i],overwrite=True)
        im.close()

In [ ]:
# Program to subtract RF-off images. 
# This version only looks for closeness in time, no pointing requirements. Probably fine, since we're already looking for closeness in time. 

def rf_off(directory,imlist):
    '''
    Subtracts rf-off Jupiter images from rf-on Jupiter images depending on closeness in time.
    
    Looks for closeness in time (if longer than 600 sec = not usable RF-off).
    Assumes the list you feed it has already been reduced to a single object, or is single image cube.
    Added header keyword indicating rf-off subtraction has been performed or not, along with history statement. 
    '''
    
    del_t_max = 0.00694444444 #max difference in JD for an rf-off to be usable (600 s, from Paul's thesis)
    image_file = np.loadtxt(imlist,dtype=str) #load the list of images
    
    for i in range(0,len(image_file)):
        im = fits.open(directory+image_file[i])
        if im[0].header['rfon'] == 1: #subtract image only if it's rf-on
            
            im[0].data = np.array(im[0].data,dtype=float) #load the image array. Save as floats to avoid issues with -numbers, floats

            rfon_exp = im[0].header['exposure']
            
            rfoff_list = [] # empty list to populate with nearby rf-off images
            
            for j in range(1,20): # look at 20 images on either side of image in question
                if i-j > 0: #Accounts for rf-on images at the beginning or end of 
                    #If it's an RF-off image:
                    if fits.open(directory+image_file[i-j])[0].header['rfon'] == 0:
                        #If exp time is the same as the rf-on image
                        if fits.open(directory+image_file[i-j])[0].header['exposure'] == rfon_exp:
                            rfoff_list.append(image_file[i-j])
                if i+j < len(image_file): 
                    if fits.open(directory+image_file[i+j])[0].header['rfon'] == 0:
                        #If exp time is the same as the rf-on image
                        if fits.open(directory+image_file[i+j])[0].header['exposure'] == rfon_exp:
                            rfoff_list.append(image_file[i+j])
                                                                                                
            #CHECKING TIME
                        
            #Setting up julian dates to use when finding the differences between them:
            jd_rfon = [] #making a list, because Python is rounding JD when try to save it directly to a variable
            jd_rfon.append(im[0].header['julian']) #The JD of the rf-on image
            jd_rfoff = [] #A list of the JDs of the rf-off images
            for k in range(0,len(rfoff_list)):
                jd_rfoff.append(fits.open(directory+rfoff_list[k])[0].header['julian'])
            #Make a list of the difference between the rf-on JD and rf-off JD's. 
            jd_diff = []
            for l in range(0,len(rfoff_list)):
                jd_diff.append(abs(jd_rfoff[l]-jd_rfon[0]))            
            
            #Choose the closest and 2nd closest RF-offs as the primary RF-off and secondary RF-off, respectively
            if len(jd_diff) >= 2:
                primary_del_t = min(jd_diff) #find the smallest value. that's the closest rf-off in JD
                jd_diff.remove(primary_del_t) #take out the smallest value
                secondary_del_t = min(jd_diff) #find the new smallest value
            elif len(jd_diff) == 1:
                primary_del_t = min(jd_diff)
                secondary_del_t = 20.0 # need to assign a value here or else will break later.
                print 'Only one RF-off image found.'
            elif len(jd_diff) == 0:
                print 'No nearby images found with the same exposure length. No usable RF-off image for', image_file[i]
                continue

            
            #Check that the closest image isn't farther than 600 s
            if primary_del_t > del_t_max:
                print 'Primary RF-off image is farther than the allowed del_t. No usable RF-off image for', image_file[i]
                continue #look at next RF-on image

            #Doing some janky stuff to find the RF-off image that corresponds to 'primary_del_t' and 'secondary_del_t'
            for m in range(0,len(rfoff_list)): # loop through list of rf-off images
                jdrfoff = fits.open(directory+rfoff_list[m])[0].header['julian'] #find the jd for that image
                if abs(jd_rfon[0]-jdrfoff) == primary_del_t: #find the jd for the rf off and rf on. if it matches, it's the primary image
                    primary_rfoff_image = rfoff_list[m] #The name of the RF-off image closest in time to the RF-on image
                elif abs(jd_rfon[0]-jdrfoff) == secondary_del_t:
                    secondary_rfoff_image = rfoff_list[m] #The name of the RF-off image 2nd closest in time to the RF-on image
            
            #print 'primary and secondary rf-off images',primary_rfoff_image,secondary_rfoff_image
            
                        
            if abs(primary_del_t-secondary_del_t)/primary_del_t < 0.1 and secondary_del_t <= del_t_max:
                #and abs(rfon_distance_check-primary_distance_check) <= 0.001 and abs(rfon_distance_check-secondary_distance_check) <= 0.001:
                #old pointing requirements
                #and pri_rfon_alt_diff <= 0.002 and pri_rfon_az_diff <= 0.002 and sec_rfon_alt_diff <= 0.002 and sec_rfon_az_diff <= 0.002:
                print 'The primary and secondary rf-off images are close enough in time and pointing to be averaged for image', image_file[i]
                #Make average rf-off image, using primary and secondary rf-off images:
                rfoff_ave = (fits.open(directory+primary_rfoff_image)[0].data + fits.open(directory+secondary_rfoff_image)[0].data)/2.
                #Subtract averaged rf-off:
                im[0].data -= rfoff_ave
                #Save header history
                im[0].header['history'] = 'Subtracted average of primary and secondary RF-off images, '+primary_rfoff_image+' & '+secondary_rfoff_image+', on '+str(datetime.datetime.now())
                im[0].header['RFSUB'] = (1,'RF-off sub completed? 1 = yes 0 = no')
                im[0].writeto(directory+image_file[i],overwrite=True)
                im.close()
                
            elif primary_del_t <= del_t_max:
                #and abs(rfon_distance_check-primary_distance_check) <= 0.001:
                #old pointing requirements
                #and pri_rfon_alt_diff <= 0.002 and pri_rfon_az_diff <= 0.002:
                #print 'Only the primary RF-off image is close enough in time and pointing to be subtracted for image',image_file[i]
                im[0].data -= fits.open(directory+primary_rfoff_image)[0].data
                im[0].header['history'] = 'Subtracted primary RF-off, '+primary_rfoff_image+', on '+str(datetime.datetime.now())
                im[0].header['RFSUB'] = (1,'RF-off sub completed? 1 = yes 0 = no')
                im[0].writeto(directory+image_file[i],overwrite=True)
                im.close()
                
            elif secondary_del_t <= del_t_max:
                #and abs(rfon_distance_check-secondary_distance_check) <= 0.001:
                #old pointing requirements
                #and sec_rfon_alt_diff <= 0.002 and sec_rfon_az_diff <= 0.002:
                #print 'Only the secondary RF-off image is close enough in time to be subtracted for image',image_file[i]                    
                im[0].data -= fits.open(directory+secondary_rfoff_image)[0].data
                im[0].header['history'] = 'Subtracted secondary RF-off, '+secondary_rfoff_image+', on '+str(datetime.datetime.now())
                im[0].header['RFSUB'] = (1,'RF-off sub completed? 1 = yes 0 = no')
                im[0].writeto(directory+image_file[i],overwrite=True)
                im.close()
                
            else:
                print 'None of the RF-off images are close enough. No subtraction performed on',image_file[i]
                im[0].header['RFSUB'] = (0,'RF-off sub completed? 1 = yes 0 = no')
                im[0].writeto(directory+image_file[i],overwrite=True)
                im.close()
            continue

In [ ]:
def div_by_exptime(directory,list_of_images):
    '''
    Divides images by exposure time. Only divides RF-on images, so be sure to do this step after RF-off subtractin has been completed.
    '''
    imlist = np.loadtxt(list_of_images,dtype=str)
    for i in range(0,len(imlist)):
        im = fits.open(directory+imlist[i])
        if im[0].header['rfon'] == 1:
            exptime = im[0].header['exposure']
            im[0].data = np.array(im[0].data,dtype=float)
            im[0].data /= exptime
            im[0].header['HISTORY'] = 'Image divided by exposure time.'
            im.writeto(directory+imlist[i],overwrite=True)

In [ ]:
def flat_div(data_directory,imlist,flat_directory,flatlist):
    '''
    Divides RF-on images by flats with corresponding wavelength. 
    '''
    image_list = np.loadtxt(imlist,dtype=str)
    flat_list = np.loadtxt(flatlist,dtype=str)
    
    for i in range(0,len(image_list)):
        
        im = fits.open(data_directory+image_list[i])
        im_image = im[0].data
        im_header = im[0].header
        
        if im[0].header['rfon'] == 1: 
            im_wavelength = im[0].header['lambda']
            
            check_if_already_divided = []
            #Find the flat with the correct wavelength:
            for j in range(0,len(flat_list)):
                
                flat = fits.open(flat_directory+flat_list[j])
                flat_image = flat[0].data
                flat_header = flat[0].header
                flat_wavelength = flat_header['lambda']
                
                if flat_wavelength == im_wavelength and len(check_if_already_divided) == 0 and flat_header['rfon'] == 1:
                    
                    #Divide by flat
                    im[0].data = np.array(im[0].data,dtype=float)
                    im[0].data = np.array(im[0].data)/flat_image

                    im[0].header['history'] = 'Image divided by '+flat_directory+flat_list[j]+' on '+str(datetime.datetime.now())
                    im.writeto(data_directory+image_list[i],overwrite=True) #Save over the old image
                    
                    check_if_already_divided.append(1)
                    
                else:
                    continue
        im.close()

In [ ]:
def flat_div_single_image(data_directory,image,flat_directory,flatlist):
    '''
    Same as flat_div() but for a single frame (can be used as check)
    '''
    flat_list = np.loadtxt(flatlist,dtype=str)
            
    im = fits.open(data_directory+image)
    im_image = im[0].data
    im_header = im[0].header

    if im[0].header['rfon'] == 1: 
        im_wavelength = im[0].header['lambda']

        check_if_already_divided = []
        #Find the flat with the correct wavelength:
        for j in range(0,len(flat_list)):

            flat = fits.open(flat_directory+flat_list[j])
            flat_image = flat[0].data
            flat_header = flat[0].header
            flat_wavelength = flat_header['lambda']

            if flat_wavelength == im_wavelength and len(check_if_already_divided) == 0 and flat_header['rfon'] == 1:

                #Divide by flat
                im[0].data = np.array(im[0].data,dtype=float)
                im[0].data = im[0].data/flat_image

                im[0].header['history'] = 'Image divided by '+flat_directory+flat_list[j]+' on '+str(datetime.datetime.now())
                im.writeto(data_directory+image,overwrite=True) #Save over the old image

                check_if_already_divided.append(1)
                print 'Divided.'

            else:
                continue
        im.close()

In [ ]:
def fringe_fix(directory_data,imagelist_data,directory_fringe,imagelist_fringe):
    '''
    Directory - path of location of images
    Imagelist - path+name of text file of image list
    Assumes the fringe images are fits files with their corresponding wavelength in the header. Divides data by the fringing image.
    Be careful to make sure the wavelengths are the same!!
    '''
    imlist_data = np.loadtxt(imagelist_data,dtype=str)
    imlist_fringe = np.loadtxt(imagelist_fringe,dtype=str)

    for i in range(0,len(imlist_data)):
        im_data = fits.open(directory_data+imlist_data[i])
        if im_data[0].header['rfon'] == 1:
            # Find the matching fringe image
            for j in range(0,len(imlist_fringe)):
                im_fringe = fits.open(directory_fringe+imlist_fringe[j])
                if round(im_fringe[0].header['lambda'],3) == im_data[0].header['lambda']: # assuming the wavelength is in the header
                    # If wavelengths match, divide data image by fringing image
                    print 'Fixing fringing for wavelength ',im_data[0].header['lambda']
                    div = np.array(im_data[0].data)/np.array(im_fringe[0].data)
                    im_data[0].data = div
                    # Add keyword saying its been divided already
                    im_data[0].header['history'] = 'Fringing has been corrected'
                    # Save image
                    im_data.writeto(directory_data+imlist_data[i],overwrite=True) 
                    
                im_fringe.close()
                
        im_data.close()

In [ ]:
def gaussian_filt(directory,imagelist):
    '''
    Gaussian Filter replacement for noisy flats. Makes low- and high-pass filters, and if flat is too noisy, replaces it with low-pass filter.
    Save noisy flats as their low-pass filter (based on 0.04 sigma cutoff that Paul used)
    '''
    imlist = np.loadtxt(imagelist,dtype=str)
    
    # Get rid of negative values in flats
    for i in range(0,len(imlist)):
        im = fits.open(directory+imlist[i])
        im[0].data = abs(im[0].data)
        im.writeto(directory+imlist[i],overwrite=True)
        im.close()
    
    # Calculate the std dev of a 30 pixel FWHM (Paul used 30 pixels)
    FWHM = 30
    stdev = FWHM/2.355 # from gaussian formula

    for i in range(0,len(imlist)-130): # Ignore longer wavelength images. Applying a gaussian filter to those will be bad.
        im = fits.open(directory+imlist[i])
        if im[0].header['rfon'] == 1:

            # Make low-pass filter:
            low_pass = filters.gaussian_filter(im[0].data,stdev)

            # Make high-pass filter:
            high_pass = im[0].data - filters.gaussian_filter(im[0].data,stdev)

            # Define noise in high-pass filter:
            #std.append(np.std(im[0].data-filters.gaussian_filter(im[0].data,stdev))/np.median(im[0].data))
            #wl.append(im[0].header['lambda'])

            if np.std(high_pass)/np.median(im[0].data) > 0.04: # Normalize std by median value
                im[0].data = low_pass
                im[0].header['history'] = 'Flat replaced by gaussian low pass filter'
                im.writeto(directory+imlist[i],overwrite=True)
                im.close()
            else:
                im.close()
                continue

### Compilations of programs

In [ ]:
def flat_prep(flatlist,flatdirectory):
    
    '''
    Prepares flat cube for use with Jupiter/cal star images. Order of inquiries aren't necessarily the order they're ran in.
    '''
    
    wl_correction_query = raw_input('Is this dataset from before January 2018? (if y, will corect wavelength)? y or n')
    ds_corr_query = raw_input('Do you want to correct for the Doppler shift resulting from the moition of the planet? y or n ')
    
    pix_correct_query = raw_input('Do you want to correct for hot pixels and cosmic rays? y or n ')
    if pix_correct_query == 'y':
        hot_pixel_mask_query = raw_input('Does this dataset have a hot pixel mask made already (if y will correct for hot/bad pixels, if n will make new one based on flat list and then correct for hot/bad pixels)? y or n')
        if hot_pixel_mask_query == 'y':
            pixelmaskname = raw_input('What is the path+name of the pixel mask?')
        else:
            pixelmaskname = raw_input('What do you want the path+name of the pixel mask to be?')

    fringe_query = raw_input('Correct for fringing? y or n')
    if fringe_query == 'y':
        fringedirectory = raw_input('What is the path of the location of fringing frames?')
        fringeimagelist = raw_input('What is the path+name of the list of fringing frames? ')
    
    rfoff_query_flat = raw_input('Would you like to RF-off subtract flats? y or n ')
    exptime_divide_query = raw_input('Do you want to divide by exposure time? y or n ')
    gaussian_filter_query = raw_input('Would you like to replace low-SNR flats with a Gaussian filter? y or n')
    
    
    print ' '
    print 'RUNNING PIPELINE . . . '
    print ' '

    
    # Step 1: Wavelength Correction
    if wl_correction_query == 'y':
        print 'Correcting wavelength...'
        #If wavelength needs to be corrected since it was taken before Jan. 2018
        #correct flats
        wavelength_corrector(flatlist,flatdirectory)
    elif wl_correction_query == 'n':
        print 'Not correcting wavelength.'
    
    if ds_corr_query == 'y':
        print 'Correcting wavelength for Doppler shift...'
        doppler_shift_corr(sciencelist,sciencedirectory)
    else:
        print 'Not correcting for Doppler shift.'
    
    
    # Step 2: make pixel mask, or not
    if pix_correct_query == 'y':
        if hot_pixel_mask_query == 'y':
            print 'Using '+str(pixelmaskname)
        else:
            #If need to make pixel mask, make it from flat list
            print 'Making pixel mask...'
            pix_mask_maker(flatlist, flatdirectory,pixelmaskname)
            
    # Step 3: Hot pixel correction in rf-off images
        print 'Correcting pixels and cosmic rays in RF-off images...'
        #Once hot pixel mask exists and has been saved to pixelmaskname, use it to correct for hot pixels AND correct for cosmic rays for rf-off images:
        pixel_corrector(flatdirectory,flatlist,pixelmaskname,0) #flats
        
    else:
        print 'Not correcting hot pixels or cosmic rays in RF-off images.'
    
    
    # Step 4: Subtract RF-off images
    if rfoff_query_flat == 'y':
        print 'RF-off subtracting flats...'
        # Next, flats 
        rf_off(flatdirectory,flatlist)
    else:
        print 'Not RF-off subtracting flats.'
    
    # Step 5: correct rf-on images
    if pix_correct_query == 'y':
        print 'Correcting pixels and cosmic rays in RF-on flats...'
        pixel_corrector(flatdirectory,flatlist,pixelmaskname,1) #flats
    else:
        print 'Not correcting pixels or cosmic rays in RF-on images.'
    
    # Step 6: Divide by exposure time
    if exptime_divide_query == 'y':
        # Flats:
        print 'Dividing flats by exposure time...'
        div_by_exptime(flatdirectory,flatlist)
    
    # Step 7: Correct for fringing
    if fringe_query == 'y':
        print 'Correcting for fringing.'
        fringe_fix(flatdirectory,flatlist,fringedirectory,fringeimagelist)
    else:
        print 'Not correcting for fringing.'
    
    # Step 8: Apply Gaussian filter
    if gaussian_filter_query == 'y':
        print 'Replacing low-SNR filters with Gaussian filter.'
        # Fix with filters:
        gaussian_filt(flatdirectory,flatlist)
    else:
        print 'Not replacing low-SNR flats with filter.'
    
    
    
    print 'Flat prep complete.'

In [ ]:
# Example input:
flat_prep('/Volumes/external_hd/march2017/flatlist','/Volumes/external_hd/march_2017_newfringeframes/flats/')

In [ ]:
def reduce_datacube(sciencelist,sciencedirectory,flatlist,flatdirectory,eph_file,apo_log):
    
    '''
    #Calling all programs to reduce science data (not specific to Jupiter)

    # ONLY CORRECTS SCIENCE IMAGES, NOT FLATS. 
    # CORRECT FLATS FIRST WITH flat_prep()

    #Format of variables:
    #sciencelist = '/directory/path/nameoflist'
    #sciencedirectory = '/directory/path/'
    #flatlist = '/directory/path/nameoflist'
    #flatdirectory = '/directory/path/'
    #eph_file = '/directory/path/name_of_ephemeris_file'
    #apo_log = '/directory/path/name_of_tcc_file'

    #Before using this program:
    #-Finalize science image list. This means check for saturation, repeats, aberrations before running this program.
    #-Double check TCC files. Do they have the correct number of columns?
    #-Double check eph files. Do they include RA/dec or not?
    '''
    
    
    print 'MAKE SURE FLATS HAVE ALREADY BEEN PREPARED.'
    
    
    wl_correction_query = raw_input('Is this dataset from before January 2018? (if y, will corect wavelength) y or n')
    ds_corr_query = raw_input('Do you want to correct for the Doppler shift resulting from the moition of the planet? y or n (Keep in mind wavelength of fringing images)')
    
    eph_attach_query = raw_input('Would you like to attach ephemerides and pointing info? y or n ')
    
    pix_correct_query = raw_input('Do you want to correct for hot pixels and cosmic rays? y or n ')
    if pix_correct_query == 'y':
        hot_pixel_mask_query = raw_input('Does this dataset have a hot pixel mask made already (if y will correct for hot/bad pixels, if n will make new one based on flat list and then correct for hot/bad pixels)? y or n')
        if hot_pixel_mask_query == 'y':
            pixelmaskname = raw_input('What is the path+name of the pixel mask?')
        else:
            pixelmaskname = raw_input('What do you want the path+name of the pixel mask to be?')
    
    rfoff_query = raw_input('Would you like to RF-off subtract science images? y or n ')
    
    exptime_divide_query = raw_input('Do you want to divide SCIENCE IMAGES by exposure time? y or n ')
    
    fringe_query = raw_input('Correct for fringing? y or n')
    if fringe_query == 'y':
        fringedirectory = raw_input('What is the path of the location of fringing frames?')
        fringeimagelist = raw_input('What is the path+name of the list of fringing frames? ')
    
    flat_query = raw_input('Do you want to divide by flats? y or n ')

    print ' '
    print 'RUNNING PIPELINE . . . '
    print ' '
    
    #STEP 1: Correct wavelength (or not)
    
    if wl_correction_query == 'y':
        print 'Correcting wavelength...'
        #If wavelength needs to be corrected since it was taken before 
        #correct science images
        wavelength_corrector(sciencelist,sciencedirectory)
    elif wl_correction_query == 'n':
        print 'Not correcting wavelength.'
        
    if ds_corr_query == 'y':
        print 'Correcting wavelength for Doppler shift...'
        doppler_shift_corr(sciencelist,sciencedirectory)
    else:
        print 'Not correcting for Doppler shift.'
        
        
    # STEP 2: Attach ephemerides
    
    if eph_attach_query == 'y':
        print 'Attaching ephimerides...'
        attach_eph(sciencedirectory,sciencelist,eph_file)
        print 'Attaching APO pointing data...'
        attach_apo(sciencedirectory,sciencelist,apo_log)
    else:
        print 'Not attaching ephemerides and pointing info.'
    
    
    # STEP 3: Make hot pixel mask (if need to) and/or correct for hot pixels in rf-off images
    
    # make pixel mask, or not
    if pix_correct_query == 'y':
        if hot_pixel_mask_query == 'y':
            print 'Using '+str(pixelmaskname)
        else:
            #If need to make pixel mask, make it from flat list
            print 'Making pixel mask...'
            pix_mask_maker(sciencelist, sciencedirectory,pixelmaskname)
            
    # Hot pixel correction in rf-off images
        print 'Correcting pixels and cosmic rays in RF-off images...'
        #Once hot pixel mask exists and has been saved to pixelmaskname, use it to correct for hot pixels AND correct for cosmic rays for rf-off images:
        pixel_corrector(sciencedirectory,sciencelist,pixelmaskname,0) #flats
        
    else:
        print 'Not correcting hot pixels or cosmic rays in RF-off images.'
    
    
    # STEP 4: Once rf-off images are prepared, subtract them from rf-on images
    if rfoff_query == 'y':
        print 'RF-off subtracting science images...'
        # First, science images
        rf_off(sciencedirectory,sciencelist)
    else:
        print 'Not RF-off subtracting science images.'
    
    
    # STEP 5: Correct subtracted RF-on images for cosmic rays and hot pixels
    if pix_correct_query == 'y':
        print 'Correcting pixels and cosmic rays in RF-on images...'
        pixel_corrector(sciencedirectory,sciencelist,pixelmaskname,1) #science images
    else:
        print 'Not correcting pixels or cosmic rays in RF-on images.'
    
    
    # STEP 6: Divide science images by exposure time (to prep for flat division)
    if exptime_divide_query == 'y':
        print 'Dividing science images by exposure time...'
        div_by_exptime(sciencedirectory,sciencelist)
    
    # STEP 7: Correct for fringing
    if fringe_query == 'y':
        print 'Correcting for fringing.'
        fringe_fix(sciencedirectory,sciencelist,fringedirectory,fringeimagelist)
    else:
        print 'Not correcting for fringing.'
    
    # STEP 8: Flat divide
    if flat_query == 'y':
        print 'Flat dividing...'
        # Note: This flat division program does not use any fringe correction. Just matches flats by wavelength. 
        flat_div(sciencedirectory,sciencelist,flatdirectory,flatlist)
    else:
        print 'Not dividing by flats.'
    
    print 'Pipeline complete! Images have been eph-attached, RF-off subbed, exp time divided, flat divided.'
    
    print 'Be sure to manually look through images to make sure they look okay, that there\'s no faulty RF-off subtraction, etc.'

In [ ]:
# input example:
reduce_datacube('/Users/dahlek/Desktop/may2019/imlist','/Users/dahlek/Desktop/may2019/','/Volumes/external_hd//march2017/flatlist','/Volumes/external_hd/march2017/redo/flats/','/Users/dahlek/Desktop/march2017/eph_march2017','/Users/dahlek/Desktop/march2017/170327_tel.txt')

In [ ]:
'''CAL STAR VERSION'''
#sciencelist = path+name of calstar cube
#sciencedirectory = path of calstar data location
# this version doesn't query you at the beginning, so have to wait for queries as pipeline progresses

def reduce_datacube_calstar(sciencelist,sciencedirectory,flatlist,flatdirectory):
    
    #STEP 1: Correct wavelength (or not)
    
    wl_correction_query = raw_input('Is this dataset from before January 2018? (if y, will corect wavelength) y or n')
    ds_corr_query = raw_input('Do you want to correct for the Doppler shift resulting from the moition of the planet? y or n ')
    
    if wl_correction_query == 'y':
        print 'Correcting wavelength...'
        #If wavelength needs to be corrected since it was taken before 
        #correct science images
        wavelength_corrector(sciencelist,sciencedirectory)
    elif wl_correction_query == 'n':
        print 'Not correcting wavelength.'
        
    if ds_corr_query == 'y':
        print 'Correcting wavelength for Doppler shift...'
        doppler_shift_corr(sciencelist,sciencedirectory)
    else:
        print 'Not correcting for Doppler shift.'
        
        
    # STEP 2: Attach ephemerides
    
    print 'Not attaching ephemerides and pointing info for calstars.'
    
    
    # STEP 3: Make hot pixel mask (if need to) and/or correct for hot pixels in rf-off images
    
    pix_correct_query = raw_input('Do you want to correct for hot pixels and cosmic rays? y or n ')
    if pix_correct_query == 'y':
        hot_pixel_mask_query = raw_input('Does this dataset have a hot pixel mask made already (if y will correct for hot/bad pixels, if n will make new one based on flat list and then correct for hot/bad pixels)? y or n')
        if hot_pixel_mask_query == 'y':
            pixelmaskname = raw_input('What is the path+name of the pixel mask?')
        else:
            #If need to make pixel mask, make it from flat list
            pixelmaskname = raw_input('What do you want the path+name of the pixel mask to be?')
            print 'Making pixel mask...'
            pix_mask_maker(flatlist, flatdirectory,pixelmaskname)

        print 'Correcting pixels and cosmic rays in RF-off images...'
        #Once hot pixel mask exists and has been saved to pixelmaskname, use it to correct for hot pixels AND correct for cosmic rays for rf-off images:
        pixel_corrector(sciencedirectory,sciencelist,pixelmaskname,0) #science images
        
        print 'Not correcting pixels and cosmic eays in RF-off flats.'
        #pixel_corrector(flatdirectory,flatlist,pixelmaskname,0) #flats
    
    else:
        print 'Not correcting hot pixels or cosmic rays in RF-off images.'
    
    # STEP 4: Once rf-off images are prepared, subtract them from rf-on images
    
    rfoff_query = raw_input('Would you like to RF-off subtract science images? y or n ')
    if rfoff_query == 'y':
        print 'RF-off subtracting science images...'
        # First, science images
        rf_off(sciencedirectory,sciencelist)
    else:
        print 'Not RF-off subtracting science images.'
    
    
    # STEP 5: Correct subtracted RF-on images for cosmic rays and hot pixels
    
    if pix_correct_query == 'y':
        print 'Correcting pixels and cosmic rays in RF-on images...'
        pixel_corrector(sciencedirectory,sciencelist,pixelmaskname,1) #science images
        print 'Not correcting pixels and cosmic eays in RF-on flats.'
        #pixel_corrector(flatdirectory,flatlist,pixelmaskname,1) #flats
    else:
        print 'Not correcting pixels or cosmic rays in RF-on images.'
    
    
    # STEP 6: Divide flats and science images by exposure time (to prep for flat division)
    # Science images:
    exptime_divide_query = raw_input('Do you want to divide SCIENCE IMAGES by exposure time? y or n ')
    if exptime_divide_query == 'y':
        print 'Dividing science images by exposure time...'
        div_by_exptime(sciencedirectory,sciencelist)
    
    # STEP 7: Flat divide
    flat_query = raw_input('Do you want to divide by flats? y or n ')
    if flat_query == 'y':
        print 'Flat dividing...'
        # Note: This flat division program does not use any fringe correction. Just matches flats by wavelength. 
        flat_div(sciencedirectory,sciencelist,flatdirectory,flatlist)
    else:
        print 'Not dividing by flats.'
    
    print 'Pipeline complete! Images have been eph-attached, RF-off subbed, exp time divided, flat divided.'
    print 'Be sure to manually look through images to make sure they look okay, that there\'s no faulty RF-off subtraction, etc.'

In [ ]:
'''CAL STAR VERSION, WITH QUERIES ANSWERED for MARCH 2017'''
#sciencelist = path+name of calstar cube
#sciencedirectory = path of calstar data location

#Queries answered:
# y to correcting wavelength
# y to correcting hot pixels
# y to already having a mask
# pixel mask: /Users/dahlek/Desktop/march2017/pix_mask


def reduce_datacube_calstar_queriesanswered(sciencelist,sciencedirectory,flatlist,flatdirectory):
    
    #STEP 1: Correct wavelength (or not)
    
    wl_correction_query = 'y' 
    #raw_input('Is this dataset from before January 2018 (Does the wavelength need to be corrected)? y or n')
    #ds_corr_query = raw_input('Do you want to correct for the Doppler shift resulting from the moition of the planet? y or n ')
    ds_corr_query = 'n'
    
    if wl_correction_query == 'y':
        print 'Correcting wavelength...'
        #If wavelength needs to be corrected since it was taken before 
        #correct science images
        wavelength_corrector(sciencelist,sciencedirectory)
        #correct flats
        wavelength_corrector(flatlist,flatdirectory)
    elif wl_correction_query == 'n':
        print 'Not correcting wavelength.'
        
    if ds_corr_query == 'y':
        print 'Correcting wavelength for Doppler shift...'
        doppler_shift_corr(sciencelist,sciencedirectory)
    else:
        print 'Not correcting for Doppler shift.'
        
    # STEP 2: Attach ephemerides
    print 'Not attaching ephemerides and pointing info for calstars.'
    
    
    # STEP 3: Make hot pixel mask (if need to) and/or correct for hot pixels in rf-off images
    pix_correct_query = 'y'
    #raw_input('Do you want to correct for hot pixels and cosmic rays? y or n ')
    if pix_correct_query == 'y':
        hot_pixel_mask_query = 'y'
        #raw_input('Does this dataset have a hot pixel mask made already (if y will correct for hot/bad pixels, if n will make new one based on flat list and then correct for hot/bad pixels)? y or n')
        if hot_pixel_mask_query == 'y':
            pixelmaskname = '/Users/dahlek/Desktop/march2017/pix_mask_march2017' #raw_input('What is the path+name of the pixel mask?')
        else:
            #If need to make pixel mask, make it from flat list
            pixelmaskname = raw_input('What do you want the path+name of the pixel mask to be?')
            print 'Making pixel mask...'
            pix_mask_maker(flatlist, flatdirectory,pixelmaskname)

        print 'Correcting pixels and cosmic rays in RF-off images...'
        #Once hot pixel mask exists and has been saved to pixelmaskname, use it to correct for hot pixels AND correct for cosmic rays for rf-off images:
        pixel_corrector(sciencedirectory,sciencelist,pixelmaskname,0) #science images
        
        print 'Not correcting pixels and cosmic eays in RF-off flats.'
        #pixel_corrector(flatdirectory,flatlist,pixelmaskname,0) #flats
    
    else:
        print 'Not correcting hot pixels or cosmic rays in RF-off images.'
    
    
    # STEP 4: Once rf-off images are prepared, subtract them from rf-on images
    rfoff_query = 'y'
    #raw_input('Would you like to RF-off subtract science images? y or n ')
    if rfoff_query == 'y':
        print 'RF-off subtracting science images...'
        # First, science images
        rf_off(sciencedirectory,sciencelist)
    else:
        print 'Not RF-off subtracting science images.'
    
    
    # STEP 5: Correct subtracted RF-on images for cosmic rays and hot pixels
    if pix_correct_query == 'y':
        print 'Correcting pixels and cosmic rays in RF-on images...'
        pixel_corrector(sciencedirectory,sciencelist,pixelmaskname,1) #science images
        print 'Not correcting pixels and cosmic eays in RF-on flats.'
        #pixel_corrector(flatdirectory,flatlist,pixelmaskname,1) #flats
    else:
        print 'Not correcting pixels or cosmic rays in RF-on images.'
    
    
    
    # STEP 6: Divide flats and science images by exposure time (to prep for flat division)
    # Science images:
    exptime_divide_query = 'y'
    #raw_input('Do you want to divide SCIENCE IMAGES by exposure time? y or n ')
    if exptime_divide_query == 'y':
        print 'Dividing science images by exposure time...'
        div_by_exptime(sciencedirectory,sciencelist)
    exptime_divide_query_2 = 'n'
    #raw_input('Do you want to divide FLATS by exposure time? y or n ')
    if exptime_divide_query_2 == 'y':
        # Flats:
        print 'Dividing flats by exposure time...'
        div_by_exptime(flatdirectory,flatlist)
    
    
    # STEP 7: Flat divide
    flat_query = 'y'
    #raw_input('Do you want to divide by flats? y or n ')
    if flat_query == 'y':
        print 'Flat dividing...'
        # Note: This flat division program does not use any fringe correction. Just matches flats by wavelength. 
        flat_div(sciencedirectory,sciencelist,flatdirectory,flatlist)
    else:
        print 'Not dividing by flats.'
    
    print 'Pipeline complete! Images have been eph-attached, RF-off subbed, exp time divided, flat divided.'
    print 'Be sure to manually look through images to make sure they look okay, that there\'s no faulty RF-off subtraction, etc.'

In [ ]:
# Example input; would have to reduce each calstar cube by itself
reduce_datacube_calstar('/Users/dahlek/Desktop/march2017/stars/star1','/Users/dahlek/Desktop/march2017/stars/','/Users/dahlek/Desktop/march2017/flatlist','/Users/dahlek/Desktop/march2017/flats/')
reduce_datacube_calstar_queriesanswered('/Users/dahlek/Desktop/march2017/stars/star2','/Users/dahlek/Desktop/march2017/stars/','/Users/dahlek/Desktop/march2017/flatlist','/Users/dahlek/Desktop/march2017/flats/')
reduce_datacube_calstar_queriesanswered('/Users/dahlek/Desktop/march2017/stars/star3','/Users/dahlek/Desktop/march2017/stars/','/Users/dahlek/Desktop/march2017/flatlist','/Users/dahlek/Desktop/march2017/flats/')
reduce_datacube_calstar_queriesanswered('/Users/dahlek/Desktop/march2017/stars/star5','/Users/dahlek/Desktop/march2017/stars/','/Users/dahlek/Desktop/march2017/flatlist','/Users/dahlek/Desktop/march2017/flats/')
reduce_datacube_calstar_queriesanswered('/Users/dahlek/Desktop/march2017/stars/star6','/Users/dahlek/Desktop/march2017/stars/','/Users/dahlek/Desktop/march2017/flatlist','/Users/dahlek/Desktop/march2017/flats/')
reduce_datacube_calstar_queriesanswered('/Users/dahlek/Desktop/march2017/stars/star7','/Users/dahlek/Desktop/march2017/stars/','/Users/dahlek/Desktop/march2017/flatlist','/Users/dahlek/Desktop/march2017/flats/')
reduce_datacube_calstar_queriesanswered('/Users/dahlek/Desktop/march2017/stars/star8','/Users/dahlek/Desktop/march2017/stars/','/Users/dahlek/Desktop/march2017/flatlist','/Users/dahlek/Desktop/march2017/flats/')